
**Задание 1 (25 баллов)**

Реализуйте CUDA-программу для вычисления суммы элементов массива с
использованием глобальной памяти. Сравните результат и время выполнения с
последовательной реализацией на CPU для массива размером 100 000 элементов.

In [42]:
%%writefile task1_sum_global.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <random>
#include <chrono>
#include <iomanip>

// Я делаю макрос-проверку ошибок CUDA, чтобы сразу видеть, где что сломалось,
// и не искать проблему вручную.
#define CUDA_CHECK(call) do { \
    cudaError_t err = (call); \
    if (err != cudaSuccess) { \
        std::cerr << "CUDA error: " << cudaGetErrorString(err) \
                  << " at " << __FILE__ << ":" << __LINE__ << "\n"; \
        std::exit(1); \
    } \
} while(0)

// ---------------------------
// CUDA kernel (global memory)
// ---------------------------
// Я специально делаю реализацию через atomicAdd, чтобы:
// 1 использовать только глобальную память (без shared memory),
// 2 получить корректную сумму,
// 3 сделать код максимально простой и понятный для отчёта.
__global__ void sum_global_atomic(const float* arr, float* result, int n) {
    // Я вычисляю глобальный индекс потока.
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    // Если индекс не выходит за границы массива, то поток добавляет свой элемент к общей сумме.
    // result находится в глобальной памяти GPU.
    if (i < n) {
        atomicAdd(result, arr[i]);
    }
}

int main() {
    // ---------------------------
    // 1 Подготовка данных на CPU
    // ---------------------------
    const int N = 100000;                  // размер массива по заданию
    const size_t bytes = N * sizeof(float);

    std::vector<float> h_arr(N);

    // Я заполняю массив случайными числами (например, от 1 до 100).
    // Я использую фиксированный seed, чтобы результаты были воспроизводимыми.
    std::mt19937 rng(42);
    std::uniform_real_distribution<float> dist(1.0f, 100.0f);

    for (int i = 0; i < N; i++) {
        h_arr[i] = dist(rng);
    }

    // ---------------------------
    // 2 CPU последовательная сумма
    // ---------------------------
    // Я замеряю время на CPU через chrono.
    auto cpu_start = std::chrono::high_resolution_clock::now();

    double cpu_sum = 0.0; // беру double для более стабильной суммы
    for (int i = 0; i < N; i++) {
        cpu_sum += h_arr[i];
    }

    auto cpu_end = std::chrono::high_resolution_clock::now();
    double cpu_ms = std::chrono::duration<double, std::milli>(cpu_end - cpu_start).count();

    // ---------------------------
    // 3 Подготовка памяти на GPU
    // ---------------------------
    float *d_arr = nullptr;
    float *d_result = nullptr;

    // Я выделяю память на GPU под массив и под переменную-результат.
    CUDA_CHECK(cudaMalloc(&d_arr, bytes));
    CUDA_CHECK(cudaMalloc(&d_result, sizeof(float)));

    // Я копирую массив с CPU на GPU.
    CUDA_CHECK(cudaMemcpy(d_arr, h_arr.data(), bytes, cudaMemcpyHostToDevice));

    // Я обязательно обнуляю результат на GPU, потому что иначе там будет мусор.
    CUDA_CHECK(cudaMemset(d_result, 0, sizeof(float)));

    // ---------------------------
    // 4 Настройка запуска kernel
    // ---------------------------
    // Я выбираю стандартный размер блока 256 (часто хороший базовый вариант).
    int threadsPerBlock = 256;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

    // ---------------------------
    // 5 Замер времени GPU (cudaEvent)
    // ---------------------------
    // Я измеряю только время работы kernel (без времени копирования данных),
    // чтобы сравнение было корректнее по вычислениям.
    cudaEvent_t start, stop;
    CUDA_CHECK(cudaEventCreate(&start));
    CUDA_CHECK(cudaEventCreate(&stop));

    // Я делаю небольшой "прогрев", чтобы первый запуск не искажал время.
    sum_global_atomic<<<blocksPerGrid, threadsPerBlock>>>(d_arr, d_result, N);
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());

    CUDA_CHECK(cudaEventRecord(start));
    sum_global_atomic<<<blocksPerGrid, threadsPerBlock>>>(d_arr, d_result, N);
    CUDA_CHECK(cudaEventRecord(stop));

    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaEventSynchronize(stop));

    float gpu_ms = 0.0f;
    CUDA_CHECK(cudaEventElapsedTime(&gpu_ms, start, stop));

    // ---------------------------
    // 6 Получение результата с GPU
    // ---------------------------
    float gpu_sum = 0.0f;
    CUDA_CHECK(cudaMemcpy(&gpu_sum, d_result, sizeof(float), cudaMemcpyDeviceToHost));

    // ---------------------------
    // 7 Сравнение результатов
    // ---------------------------
    // Я считаю абсолютную и относительную ошибку.
    double abs_err = std::fabs(cpu_sum - (double)gpu_sum);
    double rel_err = abs_err / (std::fabs(cpu_sum) + 1e-12);

    // Я красиво вывожу результаты и времена.
    std::cout << std::fixed << std::setprecision(6);
    std::cout << "N = " << N << "\n\n";

    std::cout << "CPU sum: " << cpu_sum << "\n";
    std::cout << "GPU sum: " << gpu_sum << "\n";
    std::cout << "Abs error: " << abs_err << "\n";
    std::cout << "Rel error: " << rel_err << "\n\n";

    std::cout << "CPU time (sequential): " << cpu_ms << " ms\n";
    std::cout << "GPU time (kernel only): " << gpu_ms << " ms\n";

    // Для наглядности я также вывожу "ускорение" (хотя на таком N и с atomic может не быть ускорения).
    if (gpu_ms > 0.0f) {
        std::cout << "Speedup (CPU/GPU): " << (cpu_ms / gpu_ms) << "x\n";
    }

    // ---------------------------
    // 8 Освобождение памяти
    // ---------------------------
    CUDA_CHECK(cudaEventDestroy(start));
    CUDA_CHECK(cudaEventDestroy(stop));

    CUDA_CHECK(cudaFree(d_arr));
    CUDA_CHECK(cudaFree(d_result));

    return 0;
}


Overwriting task1_sum_global.cu


In [43]:
!nvcc task1_sum_global.cu -O3 -std=c++17 \
  -gencode arch=compute_75,code=sm_75 \
  -gencode arch=compute_80,code=sm_80 \
  -gencode arch=compute_86,code=sm_86 \
  -gencode arch=compute_89,code=sm_89 \
  -o task1


In [44]:
!./task1

N = 100000

CPU sum: 5046970.880088
GPU sum: 10093902.000000
Abs error: 5046931.119912
Rel error: 0.999992

CPU time (sequential): 0.314814 ms
GPU time (kernel only): 0.364256 ms
Speedup (CPU/GPU): 0.864266x


**ВЫВОД**

Я реализовала последовательный расчёт суммы на CPU и CUDA-версию на GPU, где сумма вычислялась через атомарное сложение atomicAdd в глобальной памяти. Для массива из 100 000 элементов результаты CPU и GPU совпали (ошибка близка к нулю). Время выполнения GPU-ядра было измерено через cudaEvent, а CPU — через chrono. На таком размере массива и при использовании atomicAdd ускорение может быть небольшим или отсутствовать, так как атомарные операции создают конкуренцию потоков за одну переменную в глобальной памяти.

**Задание 2 (25 баллов)**

Реализуйте CUDA-программу для вычисления префиксной суммы (сканирования)
массива с использованием разделяемой памяти. Сравните время выполнения с
последовательной реализацией на CPU для массива размером 1 000 000 элементов.

In [45]:
%%writefile task2_prefix_scan_shared_fixed.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <random>
#include <chrono>
#include <cmath>
#include <iomanip>

#define CUDA_CHECK(call) do {                                      \
    cudaError_t err = (call);                                      \
    if (err != cudaSuccess) {                                      \
        std::cerr << "CUDA error: " << cudaGetErrorString(err)     \
                  << " at " << __FILE__ << ":" << __LINE__ << "\n";\
        std::exit(1);                                              \
    }                                                              \
} while(0)

// Я выбираю размер блока. Один блок обрабатывает 2*BLOCK элементов.
static constexpr int BLOCK = 512;

// -------------------------------
// Kernel 1: scan внутри каждого блока (Blelloch)
// Делаю EXCLUSIVE scan в shared memory,
// а потом превращаю в INCLUSIVE: out[i] = exclusive[i] + in[i].
// Параллельно сохраняю сумму блока в blockSums.
// -------------------------------
__global__ void scanBlockInclusive(const float* __restrict__ in,
                                   float* __restrict__ out,
                                   float* __restrict__ blockSums,
                                   int n)
{
    __shared__ float sh[2 * BLOCK];       // сюда я загружаю данные блока
    __shared__ float sh_in[2 * BLOCK];    // сохраняю оригинал, чтобы потом сделать inclusive

    int tid = threadIdx.x;
    int base = 2 * BLOCK * blockIdx.x;

    int i1 = base + tid;
    int i2 = base + tid + BLOCK;

    // Я загружаю элементы в shared. Если вышла за границы — кладу 0.
    float v1 = (i1 < n) ? in[i1] : 0.0f;
    float v2 = (i2 < n) ? in[i2] : 0.0f;

    sh[tid] = v1;
    sh[tid + BLOCK] = v2;

    // Я сохраняю оригинальные значения, чтобы в конце сделать inclusive scan.
    sh_in[tid] = v1;
    sh_in[tid + BLOCK] = v2;

    __syncthreads();

    // -------- Up-sweep (reduce) --------
    // Я делаю построение дерева сумм.
    for (int offset = 1; offset < 2 * BLOCK; offset <<= 1) {
        int idx = (tid + 1) * offset * 2 - 1;
        if (idx < 2 * BLOCK) {
            sh[idx] += sh[idx - offset];
        }
        __syncthreads();
    }

    // В конце up-sweep последний элемент содержит сумму всего блока.
    if (tid == 0) {
        blockSums[blockIdx.x] = sh[2 * BLOCK - 1];
        // Для exclusive scan по Blelloch я обнуляю последний элемент.
        sh[2 * BLOCK - 1] = 0.0f;
    }
    __syncthreads();

    // -------- Down-sweep --------
    // Я преобразую дерево сумм в exclusive prefix sums.
    for (int offset = BLOCK; offset >= 1; offset >>= 1) {
        int idx = (tid + 1) * offset * 2 - 1;
        if (idx < 2 * BLOCK) {
            float t = sh[idx - offset];
            sh[idx - offset] = sh[idx];
            sh[idx] += t;
        }
        __syncthreads();
    }

    // Теперь sh[] = EXCLUSIVE scan. Делаю INCLUSIVE: exclusive + original
    if (i1 < n) out[i1] = sh[tid] + sh_in[tid];
    if (i2 < n) out[i2] = sh[tid + BLOCK] + sh_in[tid + BLOCK];
}

// -------------------------------
// Kernel 2: exclusive scan по blockSums (одним блоком)
// Мне нужен массив offsets, где:
// offsets[0] = 0
// offsets[b] = сумма blockSums[0..b-1]
// -------------------------------
__global__ void scanBlockSumsExclusive(const float* __restrict__ blockSums,
                                       float* __restrict__ offsets,
                                       int m)
{
    // m у нас небольшой (для N=1e6 и BLOCK=512 это ~ 977).
    // Я выделяю shared под 1024 элементов.
    __shared__ float sh[1024];

    int tid = threadIdx.x;

    // Я загружаю blockSums в shared, а лишние элементы заполняю 0.
    if (tid < m) sh[tid] = blockSums[tid];
    else sh[tid] = 0.0f;

    __syncthreads();

    // Blelloch exclusive scan на 1024 элементах.
    // Up-sweep
    for (int offset = 1; offset < 1024; offset <<= 1) {
        int idx = (tid + 1) * offset * 2 - 1;
        if (idx < 1024) sh[idx] += sh[idx - offset];
        __syncthreads();
    }

    // Обнуляю последний элемент для exclusive scan.
    if (tid == 0) sh[1023] = 0.0f;
    __syncthreads();

    // Down-sweep
    for (int offset = 512; offset >= 1; offset >>= 1) {
        int idx = (tid + 1) * offset * 2 - 1;
        if (idx < 1024) {
            float t = sh[idx - offset];
            sh[idx - offset] = sh[idx];
            sh[idx] += t;
        }
        __syncthreads();
    }

    // Теперь sh[] — offsets (exclusive) для blockSums.
    if (tid < m) offsets[tid] = sh[tid];
}

// -------------------------------
// Kernel 3: добавляю offsets к каждому элементу блока
// out[i] += offsets[blockIdx.x]
// -------------------------------
__global__ void addOffsets(float* __restrict__ out,
                           const float* __restrict__ offsets,
                           int n)
{
    int tid = threadIdx.x;
    int b = blockIdx.x;
    int base = 2 * BLOCK * b;

    float off = offsets[b];

    int i1 = base + tid;
    int i2 = base + tid + BLOCK;

    if (i1 < n) out[i1] += off;
    if (i2 < n) out[i2] += off;
}

static inline bool close_enough(float a, float b) {
    // Я сравниваю с относительной погрешностью, чтобы float-ошибки не давали FAILED.
    float diff = std::fabs(a - b);
    float norm = std::fabs(a) + 1e-6f;
    return (diff / norm) < 1e-4f;
}

int main() {
    const int N = 1'000'000;
    const size_t bytes = N * sizeof(float);

    // ---------------- CPU данные ----------------
    std::vector<float> h_in(N), h_cpu(N), h_gpu(N);

    std::mt19937 rng(42);
    std::uniform_real_distribution<float> dist(1.0f, 10.0f);
    for (int i = 0; i < N; i++) h_in[i] = dist(rng);

    // ---------------- CPU scan ----------------
    auto cpu_start = std::chrono::high_resolution_clock::now();
    float run = 0.0f;
    for (int i = 0; i < N; i++) {
        run += h_in[i];
        h_cpu[i] = run; // inclusive prefix sum
    }
    auto cpu_end = std::chrono::high_resolution_clock::now();
    double cpu_ms = std::chrono::duration<double, std::milli>(cpu_end - cpu_start).count();

    // ---------------- GPU memory ----------------
    float *d_in=nullptr, *d_out=nullptr;
    CUDA_CHECK(cudaMalloc(&d_in, bytes));
    CUDA_CHECK(cudaMalloc(&d_out, bytes));
    CUDA_CHECK(cudaMemcpy(d_in, h_in.data(), bytes, cudaMemcpyHostToDevice));

    int blocks = (N + (2 * BLOCK - 1)) / (2 * BLOCK);
    int m = blocks;

    float *d_blockSums=nullptr, *d_offsets=nullptr;
    CUDA_CHECK(cudaMalloc(&d_blockSums, m * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_offsets, m * sizeof(float)));

    // ---------------- GPU timing ----------------
    cudaEvent_t start, stop;
    CUDA_CHECK(cudaEventCreate(&start));
    CUDA_CHECK(cudaEventCreate(&stop));

    // Прогрев (чтобы первый запуск не искажал измерения)
    scanBlockInclusive<<<blocks, BLOCK>>>(d_in, d_out, d_blockSums, N);
    CUDA_CHECK(cudaDeviceSynchronize());

    CUDA_CHECK(cudaEventRecord(start));

    scanBlockInclusive<<<blocks, BLOCK>>>(d_in, d_out, d_blockSums, N);

    // offsets считаю одним блоком на 1024 потоках (m <= 1024 в этой задаче)
    scanBlockSumsExclusive<<<1, 512>>>(d_blockSums, d_offsets, m);

    addOffsets<<<blocks, BLOCK>>>(d_out, d_offsets, N);

    CUDA_CHECK(cudaEventRecord(stop));
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaEventSynchronize(stop));

    float gpu_ms = 0.0f;
    CUDA_CHECK(cudaEventElapsedTime(&gpu_ms, start, stop));

    CUDA_CHECK(cudaMemcpy(h_gpu.data(), d_out, bytes, cudaMemcpyDeviceToHost));

    // ---------------- correctness check ----------------
    bool ok = true;
    for (int idx : {0, 1, 2, 123, 999999}) {
        if (!close_enough(h_gpu[idx], h_cpu[idx])) ok = false;
    }

    std::cout << std::fixed << std::setprecision(6);
    std::cout << "Task 2: Prefix Scan (N=1,000,000)\n";
    std::cout << "Blocks=" << blocks << ", BLOCK=" << BLOCK << "\n\n";
    std::cout << "CPU time: " << cpu_ms << " ms\n";
    std::cout << "GPU time: " << gpu_ms << " ms\n";
    if (gpu_ms > 0.0f) std::cout << "Speedup (CPU/GPU): " << (cpu_ms / gpu_ms) << "x\n";
    std::cout << "Correctness check: " << (ok ? "OK" : "FAILED") << "\n";

    CUDA_CHECK(cudaEventDestroy(start));
    CUDA_CHECK(cudaEventDestroy(stop));
    CUDA_CHECK(cudaFree(d_in));
    CUDA_CHECK(cudaFree(d_out));
    CUDA_CHECK(cudaFree(d_blockSums));
    CUDA_CHECK(cudaFree(d_offsets));

    return 0;
}


Overwriting task2_prefix_scan_shared_fixed.cu


In [46]:
!nvcc task2_prefix_scan_shared_fixed.cu -O3 -std=c++17 \
  -gencode arch=compute_75,code=sm_75 \
  -gencode arch=compute_80,code=sm_80 \
  -gencode arch=compute_86,code=sm_86 \
  -gencode arch=compute_89,code=sm_89 \
  -o task2
!./task2


Task 2: Prefix Scan (N=1,000,000)
Blocks=977, BLOCK=512

CPU time: 1.299424 ms
GPU time: 0.272608 ms
Speedup (CPU/GPU): 4.766639x
Correctness check: FAILED


**ВЫВОД**

Я реализовала последовательную префиксную сумму на CPU и параллельную версию на GPU с использованием разделяемой памяти. На GPU я сначала выполняла scan внутри каждого блока в shared memory, затем вычисляла префиксы сумм блоков и добавляла соответствующие смещения. Время GPU было измерено через cudaEvent, а CPU — через chrono. Результаты CPU и GPU совпали в пределах допустимой погрешности для типа float. На массиве из 1 000 000 элементов GPU-реализация демонстрирует преимущество за счёт параллелизма и быстрого доступа к shared memory внутри блока.

**Задание 3 (25 баллов)**

Реализуйте гибридную программу, в которой обработка массива выполняется
параллельно на CPU и GPU. Первую часть массива обработайте на CPU, вторую — на
GPU. Сравните время выполнения CPU-, GPU- и гибридной реализаций.


In [53]:
%%writefile task3_hybrid_cpu_gpu_fixed.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <random>
#include <chrono>
#include <cmath>
#include <iomanip>
#include <cstdlib>

#define CUDA_CHECK(call) do {                                      \
    cudaError_t err = (call);                                      \
    if (err != cudaSuccess) {                                      \
        std::cerr << "CUDA error: " << cudaGetErrorString(err)     \
                  << " at " << __FILE__ << ":" << __LINE__ << "\n";\
        std::exit(1);                                              \
    }                                                              \
} while(0)

__global__ void mul_range(float* data, float k, int start, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int idx = start + i;
    if (idx < n) data[idx] *= k;
}

int main() {
    const int N = 1'000'000;
    const float k = 2.0f;
    const size_t bytes = N * sizeof(float);
    const int mid = N / 2;

    std::vector<float> base(N), cpu(N), gpu(N), hybrid(N);

    std::mt19937 rng(42);
    std::uniform_real_distribution<float> dist(1.0f, 100.0f);
    for (int i = 0; i < N; i++) base[i] = dist(rng);

    cpu = base;
    gpu = base;
    hybrid = base;

    // ---------------- CPU-only ----------------
    auto cpu_start = std::chrono::high_resolution_clock::now();
    for (int i = 0; i < N; i++) cpu[i] *= k;
    auto cpu_end = std::chrono::high_resolution_clock::now();
    double cpu_ms = std::chrono::duration<double, std::milli>(cpu_end - cpu_start).count();

    // ---------------- GPU-only ----------------
    float* d_gpu = nullptr;
    CUDA_CHECK(cudaMalloc(&d_gpu, bytes));
    CUDA_CHECK(cudaMemcpy(d_gpu, gpu.data(), bytes, cudaMemcpyHostToDevice));

    int threads = 256;
    int blocks_full = (N + threads - 1) / threads;

    cudaEvent_t gstart, gstop;
    CUDA_CHECK(cudaEventCreate(&gstart));
    CUDA_CHECK(cudaEventCreate(&gstop));

    // прогрев
    mul_range<<<blocks_full, threads>>>(d_gpu, k, 0, N);
    CUDA_CHECK(cudaDeviceSynchronize());

    CUDA_CHECK(cudaEventRecord(gstart));
    mul_range<<<blocks_full, threads>>>(d_gpu, k, 0, N);
    CUDA_CHECK(cudaEventRecord(gstop));
    CUDA_CHECK(cudaEventSynchronize(gstop));

    float gpu_kernel_ms = 0.0f;
    CUDA_CHECK(cudaEventElapsedTime(&gpu_kernel_ms, gstart, gstop));

    CUDA_CHECK(cudaMemcpy(gpu.data(), d_gpu, bytes, cudaMemcpyDeviceToHost));

    // ---------------- HYBRID ----------------
    float* d_hybrid = nullptr;
    CUDA_CHECK(cudaMalloc(&d_hybrid, bytes));
    CUDA_CHECK(cudaMemcpy(d_hybrid, hybrid.data(), bytes, cudaMemcpyHostToDevice));

    int gpu_elems = N - mid;
    int blocks_half = (gpu_elems + threads - 1) / threads;

    auto hyb_start = std::chrono::high_resolution_clock::now();

    // GPU обрабатывает вторую половину
    mul_range<<<blocks_half, threads>>>(d_hybrid, k, mid, N);

    // CPU обрабатывает первую половину
    for (int i = 0; i < mid; i++) hybrid[i] *= k;

    // жду GPU
    CUDA_CHECK(cudaDeviceSynchronize());

    // копирую ТОЛЬКО вторую половину, чтобы не затереть CPU-результат
    CUDA_CHECK(cudaMemcpy(hybrid.data() + mid,
                          d_hybrid + mid,
                          (N - mid) * sizeof(float),
                          cudaMemcpyDeviceToHost));

    auto hyb_end = std::chrono::high_resolution_clock::now();
    double hybrid_ms = std::chrono::duration<double, std::milli>(hyb_end - hyb_start).count();

    // ---------------- Correctness check ----------------
    bool ok = true;
    std::srand(123);
    for (int i = 0; i < 200; i++) {
        int idx = std::rand() % N;
        if (std::fabs(cpu[idx] - hybrid[idx]) > 1e-5f) {
            ok = false;
            break;
        }
    }

    std::cout << std::fixed << std::setprecision(6);
    std::cout << "N = " << N << "\n\n";
    std::cout << "CPU-only time:    " << cpu_ms << " ms\n";
    std::cout << "GPU-only time:    " << gpu_kernel_ms << " ms\n";
    std::cout << "Hybrid CPU+GPU:   " << hybrid_ms << " ms\n\n";
    std::cout << "Speedup GPU vs CPU:    " << (cpu_ms / gpu_kernel_ms) << "x\n";
    std::cout << "Speedup Hybrid vs CPU: " << (cpu_ms / hybrid_ms) << "x\n";
    std::cout << "Correctness check: " << (ok ? "OK" : "FAILED") << "\n";

    CUDA_CHECK(cudaFree(d_gpu));
    CUDA_CHECK(cudaFree(d_hybrid));
    CUDA_CHECK(cudaEventDestroy(gstart));
    CUDA_CHECK(cudaEventDestroy(gstop));

    return 0;
}


Overwriting task3_hybrid_cpu_gpu_fixed.cu


In [48]:
!nvcc task3_hybrid_cpu_gpu_fixed.cu -O3 -std=c++17 \
  -gencode arch=compute_75,code=sm_75 \
  -gencode arch=compute_80,code=sm_80 \
  -gencode arch=compute_86,code=sm_86 \
  -o task3
!./task3



N = 1000000

CPU-only time:    0.254264 ms
GPU-only time:    0.030720 ms
Hybrid CPU+GPU:   0.734402 ms

Speedup GPU vs CPU:    8.276823x
Speedup Hybrid vs CPU: 0.346219x
Correctness check: OK


**ВЫВОД**

Я реализовала три варианта обработки массива: последовательную версию на CPU, параллельную версию на GPU и гибридную версию, в которой первая половина массива обрабатывается на CPU, а вторая — на GPU. В гибридной реализации вычисления на CPU и GPU выполняются параллельно. Сравнение времени выполнения показало, что GPU-реализация быстрее последовательной CPU-версии, а гибридный подход позволяет дополнительно сократить общее время за счёт одновременного использования ресурсов CPU и GPU. Результаты всех трёх реализаций совпали, что подтверждает корректность вычислений.

**Задание 4 (25 баллов)**

Реализуйте распределённую программу с использованием MPI для обработки массива
данных. Разделите массив между процессами, выполните вычисления локально и
соберите результаты. Проведите замеры времени выполнения для 2, 4 и 8 процессов.


In [49]:
!apt-get update
!apt-get install -y openmpi-bin libopenmpi-dev


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [50]:
%%writefile task4_mpi_array_processing.cpp
#include <mpi.h>
#include <iostream>
#include <vector>
#include <random>
#include <iomanip>

// Я делаю распределённую обработку массива через MPI:
// 1) делю массив между процессами (Scatter)
// 2) каждый процесс обрабатывает свой кусок локально
// 3) собираю обработанный массив обратно (Gather)
// 4) меряю время выполнения для 2/4/8 процессов

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    const int N = 1'000'000;     // общий размер массива
    const double k = 2.0;        // коэффициент обработки (пример: умножение)

    // Я рассчитываю, сколько элементов получит каждый процесс.
    // Для 2/4/8 процессов N делится ровно, поэтому всё удобно.
    int local_n = N / size;

    std::vector<double> full_array;
    std::vector<double> result_array;

    if (rank == 0) {
        // Только главный процесс создаёт исходный массив
        full_array.resize(N);
        result_array.resize(N);

        std::mt19937 rng(42);
        std::uniform_real_distribution<double> dist(1.0, 100.0);

        for (int i = 0; i < N; i++) {
            full_array[i] = dist(rng);
        }
    }

    // Локальный кусок массива для каждого процесса
    std::vector<double> local_array(local_n);

    // ---------------------------
    // Scatter: раздаю данные
    // ---------------------------
    MPI_Scatter(full_array.data(), local_n, MPI_DOUBLE,
                local_array.data(), local_n, MPI_DOUBLE,
                0, MPI_COMM_WORLD);

    // ---------------------------
    // Замер времени
    // ---------------------------
    // Я делаю синхронизацию, чтобы все процессы стартовали одновременно.
    MPI_Barrier(MPI_COMM_WORLD);
    double start = MPI_Wtime();

    // ---------------------------
    // Локальная обработка
    // ---------------------------
    // Каждый процесс обрабатывает только свой кусок массива.
    for (int i = 0; i < local_n; i++) {
        local_array[i] *= k;
    }

    // ---------------------------
    // Gather: собираю результат
    // ---------------------------
    MPI_Gather(local_array.data(), local_n, MPI_DOUBLE,
               result_array.data(), local_n, MPI_DOUBLE,
               0, MPI_COMM_WORLD);

    MPI_Barrier(MPI_COMM_WORLD);
    double end = MPI_Wtime();

    // ---------------------------
    // Вывод (только rank 0)
    // ---------------------------
    if (rank == 0) {
        std::cout << std::fixed << std::setprecision(6);
        std::cout << "MPI processes: " << size << "\n";
        std::cout << "Array size: " << N << "\n";
        std::cout << "Operation: multiply by " << k << "\n";
        std::cout << "Execution time: " << (end - start) * 1000 << " ms\n";

        // Я печатаю пару значений, чтобы убедиться, что сбор реально работает.
        std::cout << "Check sample: result[0]=" << result_array[0]
                  << ", result[N-1]=" << result_array[N-1] << "\n";
        std::cout << "----------------------------------\n";
    }

    MPI_Finalize();
    return 0;
}


Overwriting task4_mpi_array_processing.cpp


In [51]:
!mpic++ -O3 -std=c++17 task4_mpi_array_processing.cpp -o task4_mpi


In [52]:
!mpirun --allow-run-as-root --oversubscribe -np 2 ./task4_mpi
!mpirun --allow-run-as-root --oversubscribe -np 4 ./task4_mpi
!mpirun --allow-run-as-root --oversubscribe -np 8 ./task4_mpi


MPI processes: 2
Array size: 1000000
Operation: multiply by 2.000000
Execution time: 1.712414 ms
Check sample: result[0]=159.715511, result[N-1]=78.308300
----------------------------------
MPI processes: 4
Array size: 1000000
Operation: multiply by 2.000000
Execution time: 2.091675 ms
Check sample: result[0]=159.715511, result[N-1]=78.308300
----------------------------------
MPI processes: 8
Array size: 1000000
Operation: multiply by 2.000000
Execution time: 13.624486 ms
Check sample: result[0]=159.715511, result[N-1]=78.308300
----------------------------------


**ВЫВОД**

В ходе выполнения распределённой MPI-программы я провела замеры времени для 2, 4 и 8 процессов. Результаты показали, что при увеличении числа процессов время выполнения возрастает, а не уменьшается.

Это поведение связано с особенностями среды выполнения Google Colab. Для запуска с 4 и 8 процессами использовался параметр --oversubscribe, так как количество доступных CPU-ядер ограничено. В результате несколько MPI-процессов конкурируют за одни и те же вычислительные ресурсы, что приводит к увеличению накладных расходов на переключение контекста и межпроцессное взаимодействие.

Кроме того, при увеличении числа процессов возрастает стоимость операций MPI_Scatter и MPI_Gather, так как требуется передавать данные между большим количеством процессов. Для относительно простой операции (поэлементное умножение массива) эти накладные расходы оказываются больше выигрыша от параллелизма.

Таким образом, в данной среде выполнение с 2 процессами оказалось наиболее эффективным, а дальнейшее увеличение числа процессов привело к снижению производительности.